To stop processes
```bash
pkill -9 -f pw.x
pkill -9 -f gipaw.x

```

In [ ]:
import f90nml
nml = f90nml.write('sample.nml')



In [ ]:
crystal=read("test/MIN-167-350K-CuCbPyz.cif")

In [ ]:
from ase.visualize import view
view(crystal)

In [ ]:
import os

print(os.getcwd())

In [ ]:
from ase.io import read, write
from ase.visualize import view
crystal=read("../structures/KTU-183_2_auto.cif")
view(crystal)

In [ ]:
len(crystal)

# Main calculation

In [ ]:
%%time
%load_ext autoreload
%autoreload 2

from runner import pw_runner
from ase.io import read, write

import time

import os

# Defining parameters:

np_pw = 13
np_gipaw = 13

pw_params = {
    'prefix':'crystal', 
    'restart_mode' : 'from_scratch',
    'tstress':True, 
    'tprnfor':True, 
    'nosym':True, 
    'ecutwfc':10, 
    'kpts':(1, 1, 1),
    # 'kpts':None, 
    'ecutrho' : 100,
    # 'occupations' : 'smearing', 
    # 'smearing' : 'gauss', 
    # 'degauss' : 1.0e-2
}

# Loading structures:

# crystal=read("../structures/MIN-167-350K-CuCbPyz.cif")
# crystal=read("../structures/HIK-143 293K-activated.cif")
# crystal=read("../structures/HIK-143 MeOH.cif")
crystal=read("../structures/KTU-183_2_auto.cif")

tms = read("../structures/tms.xyz")
# crystal = tms
a = 10.0
tms.set_cell([a, a, a])
tms.set_cell([(a, 0, 0), (0, a, 0), (0, 0, a)])
tms.set_pbc(True)

# Creating directory to store all the data:
calc_dir = pw_runner.make_calc_dir()


print("Starting QE/SCF calculation for TMS...")
pw_runner.run_pw_scf(calc_dir, tms, num_proc_pw=np_pw, pw_params=pw_params)
print("Starting gipaw calculation for TMS...")
gipaw_out = pw_runner.run_gipaw(calc_dir, "espresso_gipaw_tms.pwo", num_proc_gipaw=np_gipaw)
chemical_shifts_iso, chemical_shifts_tensors = pw_runner.parse_gipaw_output(calc_dir + '/espresso_gipaw_tms.pwo', num_atoms=len(tms))
tms.info['chemical_shifts_iso'] = chemical_shifts_iso
tms.info['chemical_shifts_tensors'] = chemical_shifts_tensors
write(calc_dir+"/tms.xyz", tms, format='extxyz')
os.rename(calc_dir+"/espresso.pwo", calc_dir+"/espresso_tms.pwo")

print("Starting QE/SCF calculation for crystal...")
pw_runner.run_pw_scf(calc_dir, crystal, num_proc_pw=np_pw, pw_params=pw_params)
print("Starting gipaw calculation for crystal...")
gipaw_out = pw_runner.run_gipaw(calc_dir, "espresso_gipaw.pwo", num_proc_gipaw=np_gipaw)
chemical_shifts_iso, chemical_shifts_tensors = pw_runner.parse_gipaw_output(calc_dir + '/espresso_gipaw.pwo', num_atoms=len(crystal))
crystal.info['chemical_shifts_iso'] = chemical_shifts_iso
crystal.info['chemical_shifts_tensors'] = chemical_shifts_tensors
write(calc_dir+"/crystal.xyz", crystal, format='extxyz')

In [ ]:
from runner.plot import plot_chemical_shifts

path = "/media/dlb/la_cie1/repos/pw-benchmarks/qe/test/2023-01-10_17 40-57_007593"
crystal = read(path+"/crystal.xyz", format="extxyz")

plot_chemical_shifts(crystal)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = np.arange(1, len(scf_times)+1, 1, dtype=int)
plt.plot(x, scf_times_5, label='SCF time')
plt.plot(x, gipaw_times_5, label='GIPAW time')
plt.xticks(x)
plt.xlabel('Num. of MPI processes')
plt.ylabel('Time, s.')
plt.legend()
plt.show()

In [ ]:
x

In [ ]:
print(start_time, scf_time, gipaw_time-start_time)

In [ ]:
struct = read(calc_dir+"/crystal.xyz", format="extxyz")
print( struct.get_potential_energy() )
print( struct.get_forces() )
print( struct.info['chemical_shifts_iso'])
print( struct.info['chemical_shifts_tensors'])

In [ ]:
import psutil
 
# Getting % usage of virtual_memory ( 3rd field)
print('RAM memory % used:', psutil.virtual_memory()[2])
# Getting usage of virtual_memory in GB ( 4th field)
print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)

In [ ]:
os.chdir(head_dir)
crystal=read("test/MIN-167-350K-CuCbPyz.cif")

view(tms)


In [ ]:
print( crystal.get_cell() )
print( crystal.get_pbc() )

crystal=read("test/MIN-167-350K-CuCbPyz.cif")
crystal.get_forces()

In [ ]:
view(crystal)

# DFTB+

In [ ]:
# https://wiki.fysik.dtu.dk/ase/ase/calculators/dftb.html
# https://dftb.org/parameters/download

# export ASE_DFTB_COMMAND="/path/to/dftb+ > PREFIX.out"
# export DFTB_PREFIX=/path/to/mio-0-1/
import os

os.environ["ASE_DFTB_COMMAND"] = "/home/dlb/anaconda3/envs/qe/bin/dftb+"
# os.environ["DFTB_PREFIX"] = "/home/dlb/Downloads/pbc-0-3"
os.environ["DFTB_PREFIX"] = "/home/dlb/Downloads/auorgap-1-1" # doesn't work for now with crystal...


In [ ]:
from ase.calculators.dftb import Dftb
from ase.io import write, read
from ase.build import molecule
from ase.optimize import QuasiNewton

# atoms = molecule('H2O')
# crystal=read("../structures/KTU-183_2_auto.cif")
crystal=read("../structures/HIK-143 293K-activated.cif")
# crystal=read("../structures/MIN-167-350K-CuCbPyz.cif")
atoms = crystal

calc = Dftb(atoms=atoms,
            label='crystal',
            # Hamiltonian_ = "xTB",
            # Hamiltonian_Method = "GFN1-xTB",
            Hamiltonian_MaxAngularMomentum_='',
            # Hamiltonian_MaxAngularMomentum_O='p',
            # Hamiltonian_MaxAngularMomentum_H='s',
            # Hamiltonian_MaxAngularMomentum_N='s',
            # Hamiltonian_MaxAngularMomentum_C='s',
            # Hamiltonian_MaxAngularMomentum_Si='s',
            kpts=(1,1,1),
            Hamiltonian_SCC='Yes',
            # Verbosity=0,
            # Hamiltonian_OrbitalResolvedSCC = 'Yes',
            # Hamiltonian_SCCTolerance=1e-15,
            # kpts=None
            # Driver_='ConjugateGradient',
            # Driver_MaxForceComponent=1e-3,
            # Driver_MaxSteps=200,
            # Driver_LatticeOpt = 'Yes',
            # Driver_AppendGeometries = 'Yes'
            )
atoms.calc = calc

atoms.set_pbc(True)

print(atoms.get_potential_energy())

# calc.calculate(atoms)
# The 'geo_end.gen' file written by the ASE calculator
# (containing the initial geometry), has been overwritten
# by DFTB+ and now contains the final, optimized geometry.
# final = read('geo_end.gen')
# write('final.xyz', final)


# dyn = QuasiNewton(atoms, trajectory='test.traj')
# dyn.run(fmax=0.01)
# write('final.xyz', atoms)

In [ ]:
from ase import Atom, Atoms
from ase.build import bulk
from ase.calculators.lammpsrun import LAMMPS

parameters = {'pair_style': 'eam/alloy',
              'pair_coeff': ['* * NiAlH_jea.eam.alloy H Ni']}

files = ['NiAlH_jea.eam.alloy']

Ni = bulk('Ni', cubic=True)
H = Atom('H', position=Ni.cell.diagonal()/2)
NiH = Ni + H

lammps = LAMMPS(parameters=parameters, files=files)

NiH.calc = lammps
print("Energy ", NiH.get_potential_energy())

In [ ]:
! export ASE_CP2K_COMMAND="mpirun -n 2 cp2k_shell.popt"

from ase.calculators.cp2k import CP2K
from ase.build import molecule
calc = CP2K()
atoms = molecule('H2O', calculator=calc)
atoms.center(vacuum=2.0)
print(atoms.get_potential_energy())

In [ ]:
atoms.get_cell()

In [ ]:
traj = read('geo_end.xyz', index=":")
view(traj)

In [ ]:
%%time

# print(atoms.get_potential_energy())
print(atoms.get_forces())

In [ ]:
crystal.get_potential_energy()

In [ ]:
# crystal=read("../structures/HIK-143 MeOH.cif")
crystal=read("../structures/HIK-143 293K-activated.cif")

In [ ]:
atoms.get_pbc()

In [ ]:
from ase.visualize import view
view(crystal)